In [ ]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


**Setup**

Si la instalación de lightgbm da error, ejecutar en Anaconda Prompt 

`conda install -c conda-forge xgboost=1.4.0`

`conda install -c conda-forge lightgbm=3.2.1`

---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


## Prediciendo quiebra de empresas

En este Checkpoint vamos a trabajar en predecir la quiebra de empresas en Polonia. La información fue obtenida del Servicio de Información de Mercados Emergentes (EMIS por sus siglas en inglés), que es una base de datos que contiene información de los mercados emergentes alrededor del mundo. Las compañías en bancarrota fueron analizadas en el período 2000-2012, mientras que las que continuan operando se evaluaron desde 2007 a 2013.

En este caso sólo vamos a trabajar con el tercer año de los disponibles. Se puede consultar la página de UCI en donde se encuentra alojado <a href='https://archive.ics.uci.edu/ml/datasets/Polish+companies+bankruptcy+data'> aqui </a> 

El dataset empleado tiene datos de las empresas y su estatus tres años después.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as st

In [ ]:
df = pd.read_csv("../Data/3year.csv")

In [ ]:
df.head()

In [ ]:
# eliminamos la primera columna que se trata de un indice.
df = df.drop("Unnamed: 0", axis = 1)

Acá hay una descripción de los atributos

- Attr1: net profit / total assets
- Attr2: total liabilities / total assets
- Attr3: working capital / total assets
- Attr4: current assets / short-term liabilities
- Attr5: [(cash + short-term securities + receivables - short-term liabilities) / (operating expenses - depreciation)] * 365
- Attr6: retained earnings / total assets
- Attr7: EBIT / total assets
- Attr8: book value of equity / total liabilities
- Attr9: sales / total assets
- Attr10: equity / total assets
- Attr11: (gross profit + extraordinary items + financial expenses) / total assets
- Attr12: gross profit / short-term liabilities
- Attr13: (gross profit + depreciation) / sales
- Attr14: (gross profit + interest) / total assets
- Attr15: (total liabilities * 365) / (gross profit + depreciation)
- Attr16: (gross profit + depreciation) / total liabilities
- Attr17: total assets / total liabilities
- Attr18: gross profit / total assets
- Attr19: gross profit / sales
- Attr20: (inventory * 365) / sales
- Attr21: sales (n) / sales (n-1)
- Attr22: profit on operating activities / total assets
- Attr23: net profit / sales
- Attr24: gross profit (in 3 years) / total assets
- Attr25: (equity - share capital) / total assets
- Attr26: (net profit + depreciation) / total liabilities
- Attr27: profit on operating activities / financial expenses
- Attr28: working capital / fixed assets
- Attr29: logarithm of total assets
- Attr30: (total liabilities - cash) / sales
- Attr31: (gross profit + interest) / sales
- Attr32: (current liabilities * 365) / cost of products sold
- Attr33: operating expenses / short-term liabilities
- Attr34: operating expenses / total liabilities
- Attr35: profit on sales / total assets
- Attr36: total sales / total assets
- Attr37: (current assets - inventories) / long-term liabilities
- Attr38: constant capital / total assets
- Attr39: profit on sales / sales
- Attr40: (current assets - inventory - receivables) / short-term liabilities
- Attr41: total liabilities / ((profit on operating activities + depreciation) * (12/365))
- Attr42: profit on operating activities / sales
- Attr43: rotation receivables + inventory turnover in days
- Attr44: (receivables * 365) / sales
- Attr45: net profit / inventory
- Attr46: (current assets - inventory) / short-term liabilities
- Attr47: (inventory * 365) / cost of products sold
- Attr48: EBITDA (profit on operating activities - depreciation) / total assets
- Attr49: EBITDA (profit on operating activities - depreciation) / sales
- Attr50: current assets / total liabilities
- Attr51: short-term liabilities / total assets
- Attr52: (short-term liabilities * 365) / cost of products sold)
- Attr53: equity / fixed assets
- Attr54: constant capital / fixed assets
- Attr55: working capital
- Attr56: (sales - cost of products sold) / sales
- Attr57: (current assets - inventory - short-term liabilities) / (sales - gross profit - depreciation)
- Attr58: total costs /total sales
- Attr59: long-term liabilities / equity
- Attr60: sales / inventory
- Attr61: sales / receivables
- Attr62: (short-term liabilities *365) / sales
- Attr63: sales / short-term liabilities
- Attr64: sales / fixed assets
- class: toma los valores "b'0'" y "b'1'" indicando si la empresa quebro o no.

### Ejercicio 1
1) Obtener la `matriz de features X` y la `variable target y`  
2) Analizar la forma (shape) del dataset.  
3) Ver los valores que toma la variable target  
4) Aplicar LabelEncoder sobre la variable target.

In [ ]:
y = df["class"]
X = df.drop("class", axis = 1)

In [ ]:
print("el dataset tiene ", X.shape[0], " filas y ", X.shape[1], " columnas")

In [ ]:
y.unique()


In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [ ]:
y = encoder.fit_transform(y)

In [ ]:
y

### Ejercicio 2
1) Realizar un split en Train y Test, dejando un 20% para test  
2) crear un generador de CV utilizando: `cv = StratifiedKFold(n_splits=5, random_state=41, shuffle=True)` para luego pasarselo tanto a `GridSearchCV` como a `cross_val_score`


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [ ]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
cv = StratifiedKFold(n_splits=5, random_state=41, shuffle=True)

### Ejercicio 3: Xgboost
1) Importar el clasificador `XGBoost` del módulo `xgboost.sklearn` que nos provee clases y métodos aceptados por la API de sklearn. La clase que contiene este clasificador se llama XGBClassifier


In [ ]:
from xgboost.sklearn import XGBClassifier
model_xg = XGBClassifier(n_jobs=-1, use_label_encoder=False)

2) Utilizando el siguiente diccionario de parametros, optimizar los hiperparámetros.  
3) Imprimir en pantalla la mejor selección de hiperparámetros

In [ ]:
from sklearn.model_selection import GridSearchCV
params = {  
    "n_estimators": [50,100,500],      # Number of boosted trees to fit.
    "max_depth": [1,2,3,4],            # Maximum tree depth for base learners.
    "learning_rate": [0.01, 0.1],      # Boosting learning rate (xgb’s “eta”)
}

In [ ]:
xgb = GridSearchCV(model_xg, param_grid=params, cv=cv, verbose=1, n_jobs=-1)

In [ ]:
xgb.fit(X_train,y_train)

In [ ]:
xgb.best_params_

5) Evaluar el rendimiento del estimador optimo, utilizando cross validation.  
(pasarle como argumento cv el generador de CV splits del creado en el punto 3)  
**nota:** ver función evaluar_rendimiento() utilizada en la notebook `2-Boosting_Practica.ipynb`

In [ ]:
def evaluar_rendimiento(modelo, nombre, X, y, cv):
    s = cross_val_score(modelo, X, y, cv=cv, n_jobs=-1)
    print("Rendimiento de {}:\t{:0.3} ± {:0.3}".format( \
        nombre, s.mean().round(3), s.std().round(3)))

In [ ]:
evaluar_rendimiento(xgb.best_estimator_,"XG Boost", X_train, y_train, cv)


6) utilizando roc_auc_score de la libreria sklearn.metrics, determinar el valor del ROC.

In [ ]:
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)

In [ ]:
from sklearn.metrics import roc_auc_score
xgb_auc = roc_auc_score(y_test, xgb.predict(X_test))
print("El valor del AUC es: ", xgb_auc)

In [ ]:
xgb.predict(X_test)

7) utilizando plot_roc de la libreria sklearn.metrics, graficamos la curva ROC por clase

In [ ]:
from scikitplot.metrics import plot_roc
plot_roc(y_test, xgb.predict_proba(X_test), plot_micro = False, plot_macro= False);


### BONUS: 
graficar la importancia de los atributos del estimador optimizado.  

https://machinelearningmastery.com/feature-importance-and-feature-selection-with-xgboost-in-python/

In [ ]:
from xgboost import plot_importance
plot_importance(xgb.best_estimator_, height = 0.8,max_num_features=5);